In [ ]:
"""
Analyze X-A push beam in-cell absorption data to confirm resonance frequency.
"""

import h5py
import os
import numpy as np


#from entire data file, extract datasets for the runs we're interested in
absolute_path = os.path.abspath('')
rel_path = "log_2024Feb19.hdf" # in cell absorption data for push beam
filename = os.path.join(absolute_path, rel_path)
run_names = ['run17:10:54', 'run17:18:36', 'run17:24:41', 'run17:29:53'] #run names for EOM sidebands on and off

#open file and process data
with h5py.File(filename, 'r') as hdf:
    dict1 = {key: hdf[key] for key in run_names if key in hdf} #reduced dict matching run name to run
    runs = [None for _ in range(len(run_names))]
    for key in dict1:
        run = dict1[key]
        num_images = len(run)
        sorted_keys = sorted(run.keys(), key=int) #sort run keys by true numerical order (not "string numerical" order")
        
        #run_shot is a list of the shots in each run in true numerical order. each shot is one absorption trace.
        run_shot = [None for _ in range(num_images)]
        for key2 in sorted_keys:
            run_shot[int(key2)] = run[key2] 
        runs[run_names.index(key)] = run_shot

    dict2 = {key: runs for key in run_names} #dict matching run name to list of shots in each run
    print(dict2)

In [ ]:
print(dict2)